In [ ]:
!pip install streamlit pyngrok pandas numpy scikit-learn matplotlib seaborn
!pip install --upgrade pyngrok

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving employees.csv to employees (1).csv


In [ ]:
from pyngrok import ngrok
import time
ngrok.kill()
time.sleep(2)
print("All ngrok sessions terminated.")

All ngrok sessions terminated.


In [ ]:
!ngrok authtoken 30AvoTROsd1jsmIsoUdRqhq6mqa_6mfYV51RbSika5ssaDqMV # Replace with your token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
os.makedirs('/content/.streamlit', exist_ok=True)
with open('/content/.streamlit/config.toml', 'w') as f:
    f.write('''[theme]
base = "light"
primaryColor = "#9C27B0"  # Vibrant blue
backgroundColor = "#E6E6FA"  # Light blue-gray
secondaryBackgroundColor = "#B0E0E6"  # Light blue for sidebar
textColor = "#000000"  # Dark navy for text
font = "sans serif"
''')
print("Theme configuration saved to /content/.streamlit/config.toml")

Theme configuration saved to /content/.streamlit/config.toml


In [ ]:
import os
print("Directory contents:", os.listdir('/content/.streamlit'))

Directory contents: ['config.toml']


In [ ]:
import os
with open('/content/employee_salary_prediction_streamlit.py', 'w') as f:
    f.write('''import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import base64
import os

# Set page config with icon and wide layout
st.set_page_config(page_title="Employee Salary Predictor", page_icon="🧑‍💼", layout="wide")

# Custom CSS for colorful styling
st.markdown("""
<style>
    .stApp {
        background-color: #F0F4F8;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 8px;
        padding: 10px 24px;
        font-weight: bold;
    }
    .stButton>button:hover {
        background-color: #45a049;
    }
    .prediction-box {
        background-color: #2196F3;
        color: white;
        padding: 15px;
        border-radius: 10px;
        text-align: center;
        font-size: 1.2em;
        font-weight: bold;
    }
    .sidebar .sidebar-content {
        background-color: #E3F2FD;
    }
    h1, h2, h3 {
        color: #1565C0;
    }
    .stMetric label {
        color: #0D1A40;
        font-weight: bold;
    }
</style>
""", unsafe_allow_html=True)

def load_and_preprocess_data(file_path='/content/employees.csv'):
    try:
        data = pd.read_csv(file_path)
        data = data.dropna()
        le_dept = LabelEncoder()
        le_job = LabelEncoder()
        le_exp = LabelEncoder()
        data['Department'] = le_dept.fit_transform(data['Department'])
        data['Job Title'] = le_job.fit_transform(data['Job Title'])
        data['Experience Level'] = le_exp.fit_transform(data['Experience Level'])
        X = data.drop(['Employee ID', 'Name', 'Salary'], axis=1, errors='ignore')
        y = data['Salary']
        scaler = StandardScaler()
        X[['Age']] = scaler.fit_transform(X[['Age']])
        return X, y, scaler, le_dept, le_job, le_exp
    except FileNotFoundError:
        st.error("❌ Dataset 'employees.csv' not found at /content/employees.csv. Please upload it.")
        return None, None, None, None, None, None
    except Exception as e:
        st.error(f"❌ Error loading dataset: {e}")
        return None, None, None, None, None, None

def preprocess_uploaded_data(data, scaler, le_dept, le_job, le_exp):
    try:
        data = data.dropna()
        required_columns = ['Age', 'Department', 'Job Title', 'Experience Level']
        if not all(col in data.columns for col in required_columns):
            st.error("❌ Uploaded dataset must contain columns: Age, Department, Job Title, Experience Level")
            return None
        data['Department'] = le_dept.transform(data['Department'])
        data['Job Title'] = le_job.transform(data['Job Title'])
        data['Experience Level'] = le_exp.transform(data['Experience Level'])
        X = data.drop(['Employee ID', 'Name', 'Salary'], axis=1, errors='ignore')
        X[['Age']] = scaler.transform(X[['Age']])
        return X
    except ValueError as e:
        st.error(f"❌ Error processing uploaded dataset: {e}. Ensure categorical values match the training dataset: {le_dept.classes_}, {le_job.classes_}, {le_exp.classes_}")
        return None

def train_gradient_boosting(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gbr.fit(X_train, y_train)
    return gbr, X_train, X_test, y_train, y_test

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return y_pred, mse, rmse, r2

def visualize_results(y_test, y_pred, model, X):
    col1, col2 = st.columns(2)
    with col1:
        fig1, ax1 = plt.subplots(figsize=(6, 4))
        ax1.scatter(y_test, y_pred, alpha=0.5, color='#1565C0')
        ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        ax1.set_xlabel('Actual Salary')
        ax1.set_ylabel('Predicted Salary')
        ax1.set_title('Actual vs Predicted Salaries 📈')
        plt.tight_layout()
        plt.savefig('/content/salary_prediction_plot.png')
        st.pyplot(fig1)

    with col2:
        fig2, ax2 = plt.subplots(figsize=(6, 4))
        feature_importance = model.feature_importances_
        features = X.columns
        sns.barplot(x=feature_importance, y=features, ax=ax2, palette='Blues_d')
        ax2.set_title('Feature Importance 📊')
        ax2.set_xlabel('Importance')
        plt.tight_layout()
        plt.savefig('/content/feature_importance_plot.png')
        st.pyplot(fig2)

def main():
    st.title("🧑‍💼 Employee Salary Prediction ")
    st.markdown("This app predicts employee salaries using a Gradient Boosting model. Upload a dataset or input individual employee details to get predictions. 🎉")

    # Sidebar for navigation
    with st.sidebar:
        st.header("⚙️ Options")
        page = st.selectbox("Choose a page", ["Home", "Batch Prediction"], help="Navigate between single and batch predictions")

    # Load base dataset
    X, y, scaler, le_dept, le_job, le_exp = load_and_preprocess_data()
    if X is None:
        return

    # Train model
    model, X_train, X_test, y_train, y_test = train_gradient_boosting(X, y)
    y_pred, mse, rmse, r2 = evaluate_model(model, X_test, y_test)

    if page == "Home":
        st.subheader("📊 Model Performance")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Mean Squared Error", f"{mse:.2f}")
        with col2:
            st.metric("Root Mean Squared Error", f"{rmse:.2f}")
        with col3:
            st.metric("R² Score", f"{r2:.2f}")

        st.subheader("📈 Visualizations")
        visualize_results(y_test, y_pred, model, X)

        st.subheader("🔍 Predict Salary for a New Employee")
        with st.container():
            age = st.slider("Age 🧬", min_value=18, max_value=70, value=35, help="Select employee age")
            department = st.selectbox("Department 🏢", le_dept.classes_, help="Select department")
            job_title = st.selectbox("Job Title 🧑‍💼", le_job.classes_, help="Select job title")
            experience_level = st.selectbox("Experience Level 🌟", le_exp.classes_, help="Select experience level")

            if st.button("Predict Salary 🚀"):
                try:
                    department_encoded = le_dept.transform([department])[0]
                    job_title_encoded = le_job.transform([job_title])[0]
                    experience_level_encoded = le_exp.transform([experience_level])[0]
                    new_employee = pd.DataFrame({
                        'Age': [age],
                        'Department': [department_encoded],
                        'Job Title': [job_title_encoded],
                        'Experience Level': [experience_level_encoded]
                    })
                    new_employee['Age'] = scaler.transform(new_employee[['Age']])
                    predicted_salary = model.predict(new_employee)
                    st.markdown(f"<div class='prediction-box'>Predicted Salary: ${predicted_salary[0]:.2f} 💰</div>", unsafe_allow_html=True)
                except ValueError as e:
                    st.error(f"❌ Error in encoding inputs: {e}. Ensure selected options match: {le_dept.classes_}, {le_job.classes_}, {le_exp.classes_}")

    elif page == "Batch Prediction":
        st.subheader("📂 Upload Dataset for Batch Predictions")
        st.markdown("Upload a CSV file with columns: Age, Department, Job Title, Experience Level. Example:")
        st.write(pd.DataFrame({
            'Age': [35],
            'Department': [le_dept.classes_[0]],
            'Job Title': [le_job.classes_[0]],
            'Experience Level': [le_exp.classes_[0]]
        }))
        uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"], help="Ensure the file matches the training dataset structure")
        if uploaded_file:
            try:
                data = pd.read_csv(uploaded_file)
                X_uploaded = preprocess_uploaded_data(data, scaler, le_dept, le_job, le_exp)
                if X_uploaded is not None:
                    predictions = model.predict(X_uploaded)
                    data['Predicted Salary'] = predictions
                    st.write("**Predictions for Uploaded Dataset**")
                    st.dataframe(data[['Age', 'Department', 'Job Title', 'Experience Level', 'Predicted Salary']])
                    csv = data.to_csv(index=False)
                    b64 = base64.b64encode(csv.encode()).decode()
                    href = f'<a href="data:file/csv;base64,{b64}" download="predictions.csv">Download Predictions CSV 📥</a>'
                    st.markdown(href, unsafe_allow_html=True)
            except Exception as e:
                st.error(f"❌ Error processing uploaded file: {e}")

if __name__ == "__main__":
    main()
''')
print("Streamlit app code saved to /content/employee_salary_prediction_streamlit.py")

Streamlit app code saved to /content/employee_salary_prediction_streamlit.py


In [ ]:
%%writefile /content/employee_salary_prediction_streamlit.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import base64
import os

# Set page config with icon and wide layout
st.set_page_config(page_title="Employee Salary Predictor", page_icon="🧑‍💼", layout="wide")

# Custom CSS for colorful styling
st.markdown("""
<style>
    .stApp {
        background-color: #F0F4F8;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 8px;
        padding: 10px 24px;
        font-weight: bold;
    }
    .stButton>button:hover {
        background-color: #45a049;
    }
    .prediction-box {
        background-color: #2196F3;
        color: white;
        padding: 15px;
        border-radius: 10px;
        text-align: center;
        font-size: 1.2em;
        font-weight: bold;
    }
    .sidebar .sidebar-content {
        background-color: #E3F2FD;
    }
    h1, h2, h3 {
        color: #1565C0;
    }
    .stMetric label {
        color: #0D1A40;
        font-weight: bold;
    }
</style>
""", unsafe_allow_html=True)

def load_and_preprocess_data(file_path='/content/employees.csv'):
    try:
        data = pd.read_csv(file_path)
        data = data.dropna()
        le_dept = LabelEncoder()
        le_job = LabelEncoder()
        le_exp = LabelEncoder()
        data['Department'] = le_dept.fit_transform(data['Department'])
        data['Job Title'] = le_job.fit_transform(data['Job Title'])
        data['Experience Level'] = le_exp.fit_transform(data['Experience Level'])
        X = data.drop(['Employee ID', 'Name', 'Salary'], axis=1, errors='ignore')
        y = data['Salary']
        scaler = StandardScaler()
        X[['Age']] = scaler.fit_transform(X[['Age']])
        return X, y, scaler, le_dept, le_job, le_exp
    except FileNotFoundError:
        st.error("❌ Dataset 'employees.csv' not found at /content/employees.csv. Please upload it.")
        return None, None, None, None, None, None
    except Exception as e:
        st.error(f"❌ Error loading dataset: {e}")
        return None, None, None, None, None, None

def preprocess_uploaded_data(data, scaler, le_dept, le_job, le_exp):
    try:
        data = data.dropna()
        required_columns = ['Age', 'Department', 'Job Title', 'Experience Level']
        if not all(col in data.columns for col in required_columns):
            st.error("❌ Uploaded dataset must contain columns: Age, Department, Job Title, Experience Level")
            return None
        data['Department'] = le_dept.transform(data['Department'])
        data['Job Title'] = le_job.transform(data['Job Title'])
        data['Experience Level'] = le_exp.transform(data['Experience Level'])
        X = data.drop(['Employee ID', 'Name', 'Salary'], axis=1, errors='ignore')
        X[['Age']] = scaler.transform(X[['Age']])
        return X
    except ValueError as e:
        st.error(f"❌ Error processing uploaded dataset: {e}. Ensure categorical values match the training dataset: {le_dept.classes_}, {le_job.classes_}, {le_exp.classes_}")
        return None

def train_gradient_boosting(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gbr.fit(X_train, y_train)
    return gbr, X_train, X_test, y_train, y_test

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return y_pred, mse, rmse, r2

def visualize_results(y_test, y_pred, model, X):
    col1, col2 = st.columns(2)
    with col1:
        fig1, ax1 = plt.subplots(figsize=(6, 4))
        ax1.scatter(y_test, y_pred, alpha=0.5, color='#1565C0')
        ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        ax1.set_xlabel('Actual Salary')
        ax1.set_ylabel('Predicted Salary')
        ax1.set_title('Actual vs Predicted Salaries 📈')
        plt.tight_layout()
        plt.savefig('/content/salary_prediction_plot.png')
        st.pyplot(fig1)

    with col2:
        fig2, ax2 = plt.subplots(figsize=(6, 4))
        feature_importance = model.feature_importances_
        features = X.columns
        sns.barplot(x=feature_importance, y=features, ax=ax2, palette='Blues_d')
        ax2.set_title('Feature Importance 📊')
        ax2.set_xlabel('Importance')
        plt.tight_layout()
        plt.savefig('/content/feature_importance_plot.png')
        st.pyplot(fig2)

def main():
    st.title("🧑‍💼 Employee Salary Prediction ")
    st.markdown("This app predicts employee salaries using a Gradient Boosting model. Upload a dataset or input individual employee details to get predictions. 🎉")

    # Sidebar for navigation
    with st.sidebar:
        st.header("⚙️ Options")
        page = st.selectbox("Choose a page", ["Home", "Batch Prediction"], help="Navigate between single and batch predictions")

    # Load base dataset
    X, y, scaler, le_dept, le_job, le_exp = load_and_preprocess_data()
    if X is None:
        return

    # Train model
    model, X_train, X_test, y_train, y_test = train_gradient_boosting(X, y)
    y_pred, mse, rmse, r2 = evaluate_model(model, X_test, y_test)

    if page == "Home":
        st.subheader("📊 Model Performance")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Mean Squared Error", f"{mse:.2f}")
        with col2:
            st.metric("Root Mean Squared Error", f"{rmse:.2f}")
        with col3:
            st.metric("R² Score", f"{r2:.2f}")

        st.subheader("📈 Visualizations")
        visualize_results(y_test, y_pred, model, X)

        st.subheader("🔍 Predict Salary for a New Employee")
        with st.container():
            age = st.slider("Age 🧬", min_value=18, max_value=70, value=35, help="Select employee age")
            department = st.selectbox("Department 🏢", le_dept.classes_, help="Select department")
            job_title = st.selectbox("Job Title 🧑‍💼", le_job.classes_, help="Select job title")
            experience_level = st.selectbox("Experience Level 🌟", le_exp.classes_, help="Select experience level")

            if st.button("Predict Salary 🚀"):
                try:
                    department_encoded = le_dept.transform([department])[0]
                    job_title_encoded = le_job.transform([job_title])[0]
                    experience_level_encoded = le_exp.transform([experience_level])[0]
                    new_employee = pd.DataFrame({
                        'Age': [age],
                        'Department': [department_encoded],
                        'Job Title': [job_title_encoded],
                        'Experience Level': [experience_level_encoded]
                    })
                    new_employee['Age'] = scaler.transform(new_employee[['Age']])
                    predicted_salary = model.predict(new_employee)
                    st.markdown(f"<div class='prediction-box'>Predicted Salary: ${predicted_salary[0]:.2f} 💰</div>", unsafe_allow_html=True)
                except ValueError as e:
                    st.error(f"❌ Error in encoding inputs: {e}. Ensure selected options match: {le_dept.classes_}, {le_job.classes_}, {le_exp.classes_}")

    elif page == "Batch Prediction":
        st.subheader("📂 Upload Dataset for Batch Predictions")
        st.markdown("Upload a CSV file with columns: Age, Department, Job Title, Experience Level. Example:")
        st.write(pd.DataFrame({
            'Age': [35],
            'Department': [le_dept.classes_[0]],
            'Job Title': [le_job.classes_[0]],
            'Experience Level': [le_exp.classes_[0]]
        }))
        uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"], help="Ensure the file matches the training dataset structure")
        if uploaded_file:
            try:
                data = pd.read_csv(uploaded_file)
                X_uploaded = preprocess_uploaded_data(data, scaler, le_dept, le_job, le_exp)
                if X_uploaded is not None:
                    predictions = model.predict(X_uploaded)
                    data['Predicted Salary'] = predictions
                    st.write("**Predictions for Uploaded Dataset**")
                    st.dataframe(data[['Age', 'Department', 'Job Title', 'Experience Level', 'Predicted Salary']])
                    csv = data.to_csv(index=False)
                    b64 = base64.b64encode(csv.encode()).decode()
                    href = f'<a href="data:file/csv;base64,{b64}" download="predictions.csv">Download Predictions CSV 📥</a>'
                    st.markdown(href, unsafe_allow_html=True)
            except Exception as e:
                st.error(f"❌ Error processing uploaded file: {e}")

if __name__ == "__main__":
    main()

Overwriting /content/employee_salary_prediction_streamlit.py


In [ ]:
from pyngrok import ngrok
import os
import time

# Terminate any existing ngrok tunnels
ngrok.kill()
time.sleep(2)
print("All ngrok sessions terminated.")

# Start Streamlit server
get_ipython().system_raw('streamlit run /content/employee_salary_prediction_streamlit.py &')

# Create ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is live at: {public_url}")
except Exception as e:
    print(f"Error starting ngrok tunnel: {e}")

All ngrok sessions terminated.
Streamlit app is live at: NgrokTunnel: "https://42d463945707.ngrok-free.app" -> "http://localhost:8501"
